In [1]:
!pip install autogluon

  Using cached autogluon-0.7.0-py3-none-any.whl (9.7 kB)
  Using cached autogluon.multimodal-0.7.0-py3-none-any.whl (331 kB)
  Using cached autogluon.features-0.7.0-py3-none-any.whl (60 kB)
  Using cached autogluon.timeseries-0.7.0-py3-none-any.whl (108 kB)
  Using cached autogluon.core-0.7.0-py3-none-any.whl (218 kB)
  Using cached autogluon.tabular-0.7.0-py3-none-any.whl (292 kB)
  Using cached autogluon.common-0.7.0-py3-none-any.whl (45 kB)
  Using cached boto3-1.26.146-py3-none-any.whl (135 kB)
  Using cached ray-2.2.0-cp39-cp39-win_amd64.whl (20.8 MB)
  Using cached hyperopt-0.2.7-py2.py3-none-any.whl (1.6 MB)
  Using cached seqeval-1.2.2-py3-none-any.whl
  Using cached evaluate-0.3.0-py3-none-any.whl (72 kB)
  Using cached pytorch_lightning-1.9.5-py3-none-any.whl (829 kB)
  Using cached timm-0.6.13-py3-none-any.whl (549 kB)
  Using cached openmim-0.3.7-py2.py3-none-any.whl (51 kB)
  Using cached accelerate-0.16.0-py3-none-any.whl (199 kB)
  Using cached nlpaug-1.1.11-py3-none-any

In [13]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from autogluon.tabular import TabularDataset, TabularPredictor

In [3]:
train = pd.read_csv("./../DAT/train.csv")
train.head()
test = pd.read_csv("./../DAT/test.csv")

In [4]:
x_train = train.drop(['ID',"TARGET"], axis = 1)
y_train = train['TARGET']

x_test = test.drop('ID', axis = 1)

In [9]:
ordinal_features = ['요일', '범죄발생지']

for feature in ordinal_features:
    le = LabelEncoder()
    le = le.fit(x_train[feature])
    x_train[feature] = le.transform(x_train[feature])

    # x_train데이터에서 존재하지 않았던 값이 x_test 데이터에 존재할 수도 있습니다.
    # 따라서 x_test 데이터를 바로 변형시키지 않고 고윳값을 확인후 x_test 데이터를 변환합니다.
    for label in np.unique(x_test[feature]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    x_test[feature] = le.transform(x_test[feature])

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_23672\3867237117.py:11: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if label not in le.classes_:


In [12]:
train_X, val_X, train_Y, val_Y = train_test_split(x_train, y_train, test_size = 0.2, random_state = 7)

In [14]:
# autogluon 학습을 위한 데이터 형태로 변환
train = TabularDataset(train.drop(['ID'], axis=1))
test = TabularDataset(test.drop(['ID'], axis=1))

In [17]:
train

,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,안개,짙은안개,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지,TARGET
0,9,화요일,10,137,8.0,2.611124,0.000000,0.0,0.00,245.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도,2
1,11,화요일,6,438,13.0,3.209093,0.000000,0.0,0.00,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도,0
2,8,일요일,6,1729,47.0,1.619597,0.000000,0.0,0.00,40.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,인도,1
3,5,월요일,6,2337,53.0,1.921615,11.375000,0.0,0.00,225.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,주거지,1
4,9,일요일,11,1439,41.0,1.789721,0.000000,0.0,0.00,255.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,주유소,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84401,4,일요일,7,336,11.0,3.808190,99.111111,0.0,0.00,165.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,차도,1
84402,8,목요일,12,2149,38.0,1.458490,0.000000,0.0,0.00,200.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,차도,0
84403,7,일요일,6,29,46.0,2.944913,105.888889,0.0,0.00,315.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도,0
84404,1,화요일,11,536,25.0,0.493679,2.285714,8.6,10.75,330.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,주거지,1


In [18]:
predictor = TabularPredictor(label='TARGET', eval_metric='f1_macro',).fit(train)

No path specified. Models will be saved in: "AutogluonModels\ag-20230605_050001\"
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230605_050001\"
AutoGluon Version:  0.7.0
Python Version:     3.9.16
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22000
Train Data Rows:    84406
Train Data Columns: 18
Label Column: TARGET
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == int, but few unique label-values observed).
	3 unique label values:  [2, 0, 1]
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Train Data Class Count: 3
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    9010.23 MB
	Train Data (Original)  Memory Usage: 25.5

In [19]:
# 각각의 모델의 훈련 성능을 평가할 수 있음
ld_board = predictor.leaderboard(train, silent=True)

ld_board

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestEntr,0.961576,0.518673,1.868175,0.074700,11.075051,1.868175,0.074700,11.075051,1,True,7
1,KNeighborsDist,0.951829,0.381719,0.347775,0.015689,0.037938,0.347775,0.015689,0.037938,1,True,2
2,RandomForestGini,0.951196,0.527462,1.752747,0.068567,9.323822,1.752747,0.068567,9.323822,1,True,6
3,ExtraTreesEntr,0.885255,0.516834,1.693776,0.051910,5.004611,1.693776,0.051910,5.004611,1,True,10
4,ExtraTreesGini,0.881364,0.514585,1.724146,0.062485,4.851994,1.724146,0.062485,4.851994,1,True,9
5,WeightedEnsemble_L2,0.769703,0.543669,11.366937,0.413875,219.099595,0.019213,0.000000,1.963137,2,True,14
6,LightGBMLarge,0.552292,0.532819,0.419280,0.031242,1.758709,0.419280,0.031242,1.758709,1,True,13
7,KNeighborsUnif,0.547819,0.368405,0.281186,0.015557,2.489050,0.281186,0.015557,2.489050,1,True,1
8,LightGBM,0.537776,0.532459,0.474009,0.015621,1.785682,0.474009,0.015621,1.785682,1,True,5
9,CatBoost,0.527811,0.528073,0.142805,0.015620,41.418166,0.142805,0.015620,41.418166,1,True,8


In [20]:
pred = predictor.predict(test)

In [22]:
submit = pd.read_csv("./../DAT/sample_submission.csv")
submit["TARGET"] = pred
submit.head()

,ID,TARGET
0,TEST_00000,2
1,TEST_00001,0
2,TEST_00002,0
3,TEST_00003,0
4,TEST_00004,0


In [23]:
submit.to_csv("sample_submission.csv", index=False)